In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

#한글폰트 설정
import matplotlib.font_manager as fm
path = 'C:\\Users\\myksh\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumGothic.ttf'
# path = 'C:\\Users\\myksh\\AppData\\Local\\Microsoft\\Windows\\Fonts\\NanumSquare.ttf'
font_name = fm.FontProperties(fname=path).get_name()
print(font_name)
plt.rc('font', family=font_name)

plt.rcParams['font.family'] = 'NanumGothic'

#마이너스가 깨질 것을 방지
plt.rcParams['axes.unicode_minus'] = False

NanumGothic


# 데이터 로드 및 전처리

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [4]:
train = train.drop('id', axis=1)
test = test.drop('id', axis=1)

In [5]:
#연대 정리해주는 함수
def tail_year(x):
    if 0<=x<10:
        return '00'
    elif 10<=x<20:
        return '10'
    elif 20<=x<30:
        return '20'
    elif 30<=x<40:
        return '30'
    elif 40<=x<50:
        return '40'
    elif 50<=x<60:
        return '50'
    elif 60<=x<70:
        return '60'
    elif 70<=x<80:
        return '70'
    elif 80<=x<90:
        return '80'
    elif 90<=x<100:
        return '90'
def year_processing(x):
    xx = str(x)
    if xx[:2] == '18':
        return '18' + tail_year(int(xx[2:]))
    elif xx[:2] == '19':
        return '19' + tail_year(int(xx[2:]))
    elif xx[:2] == '20':
        return '20' + tail_year(int(xx[2:]))

In [6]:
#연대별로 변경
train['Year Built'] = train['Year Built'].apply(lambda x:year_processing(x))
train['Year Built'] = train['Year Built'].astype(int)

test['Year Built'] = test['Year Built'].apply(lambda x:year_processing(x))
test['Year Built'] = test['Year Built'].astype(int)

In [7]:
#연대별로 정리
train['Year Remod/Add'] = train['Year Remod/Add'].apply(lambda x:year_processing(x))
train['Year Remod/Add'] = train['Year Remod/Add'].astype(int)

test['Year Remod/Add'] = test['Year Remod/Add'].apply(lambda x:year_processing(x))
test['Year Remod/Add'] = test['Year Remod/Add'].astype(int)

In [8]:
#차고 자리 개수와 차고 면적은 의미가 비슷하므로 자리 개수를 drop
train = train.drop('Garage Cars', axis=1)
test = test.drop('Garage Cars', axis=1)

In [9]:
#2207년 데이터 삭제
train = train.drop(train[train['Garage Yr Blt']>=2022].index)

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cols = ['Exter Qual','Kitchen Qual','Bsmt Qual']
for i in cols:
    train[i] = le.fit_transform(train[i])
    test[i] = le.fit_transform(test[i])

# Modeling

In [13]:
# 대회 규칙의 평가 산식 함수를 그대로 사용
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [11]:
X = train.drop('target', axis=1)
y = train['target']

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape

((1079, 12), (270, 12))

In [17]:
from pycaret.regression import *
clf = setup(train, target='target')

,Description,Value
0,session_id,5406
1,Target,target
2,Original Data,"(1349, 13)"
3,Missing Values,False
4,Numeric Features,10
5,Categorical Features,2
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(944, 24)"


In [27]:
top5 = compare_models(sort='RMSE', n_select=3)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
ridge,Ridge Regression,19681.1355,762134443.3999,27440.3291,0.8703,0.1628,0.1215,0.0080
llar,Lasso Least Angle Regression,19652.1834,762305135.6796,27450.1181,0.8700,0.1635,0.1197,0.0080
br,Bayesian Ridge,19680.2726,762819639.5969,27452.6261,0.8702,0.1660,0.1213,0.0090
lasso,Lasso Regression,19662.4930,762570812.8311,27455.2309,0.8699,0.1633,0.1198,0.0110
lar,Least Angle Regression,19662.1659,762578563.2056,27455.5561,0.8699,0.1629,0.1198,0.0080
lr,Linear Regression,19668.5742,762956179.9848,27462.0982,0.8699,0.1629,0.1198,0.0090
rf,Random Forest Regressor,19019.2457,798936195.0744,28059.4071,0.8639,0.1566,0.1157,0.1330
gbr,Gradient Boosting Regressor,19127.0204,798986500.2989,28083.3504,0.8647,0.1550,0.1148,0.0500
et,Extra Trees Regressor,19277.2870,825374815.7028,28559.3932,0.8599,0.1561,0.1150,0.1010
lightgbm,Light Gradient Boosting Machine,19483.3267,876011718.3054,29350.9340,0.8511,0.1551,0.1148,0.0660


In [28]:
tuned_top5 = [tune_model(i) for i in top5]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,20215.2997,823513734.4754,28696.9290,0.8116,0.2499,0.1188
1,19214.9140,672193858.4525,25926.7017,0.8769,0.1291,0.1030
2,17684.3589,641202676.3367,25321.9801,0.8842,0.1345,0.0989
3,15978.0658,534932776.9990,23128.6138,0.8934,0.1270,0.0934
4,18799.0825,777808575.3098,27889.2197,0.8633,0.1441,0.1148
5,24236.6612,1092702148.8582,33056.0456,0.8184,0.2973,0.2268
6,21261.6582,986465323.8324,31408.0455,0.8876,0.1543,0.1210
7,17688.8969,546856464.5917,23384.9624,0.8929,0.1613,0.1227
8,20156.2199,804368294.0088,28361.3874,0.8969,0.1198,0.0922
9,21567.5616,748152016.9838,27352.3677,0.8767,0.1433,0.1211


In [29]:
blender_top5 = blend_models(estimator_list=tuned_top5)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,20437.0284,846151253.9432,29088.6791,0.8064,0.6865,0.1205
1,19446.3174,687870923.9197,26227.2935,0.8740,0.1276,0.1035
2,17639.6554,639357758.0463,25285.5247,0.8845,0.1326,0.0978
3,15777.1710,530579291.2342,23034.3068,0.8943,0.1250,0.0919
4,18841.1185,791306903.5516,28130.1778,0.8609,0.1451,0.1155
5,23290.6156,1038876211.9754,32231.6027,0.8274,0.2904,0.2178
6,21535.1286,997545023.3687,31583.9362,0.8863,0.1560,0.1232
7,17061.9095,530359064.4616,23029.5259,0.8961,0.1547,0.1160
8,20246.0469,844267138.3757,29056.2754,0.8918,0.1186,0.0915
9,21233.1994,744145696.5431,27279.0340,0.8774,0.1404,0.1184


In [30]:
final_model = finalize_model(blender_top5)

In [31]:
prediction = predict_model(final_model, data=X_test)
prediction['Label']

842     228679.576193
184     205120.204748
527     173167.369581
1283    194441.386561
980     256334.968933
            ...      
1218    138905.395999
1320    114835.104058
1076    160268.752058
907     260325.734135
1343    139526.251599
Name: Label, Length: 270, dtype: float64

In [32]:
NMAE(y_test, prediction['Label'])

0.10726715576510955

# 예측 및 저장

In [38]:
sub = pd.read_csv('./data/sample_submission.csv')
pred = predict_model(final_model, data=test)
sub['target'] = pred['Label']

In [41]:
sub.to_csv('./submission_data/pycaret_submission.csv', index=False)